In [5]:
import os
import pandas as pd
import requests

# Goal
Predict the daily number of cases for the given countries, regions and intervention plan.

This model is described in [From Prediction to Prescription: Evolutionary Optimization of Non-Pharmaceutical Interventions in the COVID-19 Pandemic](https://arxiv.org/abs/2005.13766) in Section 5 - Data-Driven Predictive Model.

Predictions from this model trained on the latest data are visible here: [How AI Makes Intervention Recommendations](https://evolution.ml/demos/npidashboard/). For more information see [Augmenting Human Decision Making
Optimizing COVID-19 Interventions](https://evolution.ml/esp/npi/).

## Intervention plan
An intervention plan consists in a list of containment and closure policies, as well as health system policies. See https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md for more details. The following policies are considered:

In [6]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']

## Example
For the given input, provide a model that can produce the expected output

### Given input
One row represents a country, region and day. For each row, the interventions that were in place on this day for this region and country are provided. Interventions can change from one day to the other for each country and region.

In [7]:
EXAMPLE_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"

In [8]:
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

In [9]:
prediction_input_df.head()

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Expected output
For each row that was provided in input, i.e. for each country, region and day, the output should contain an additional `PredictedDailyNewCases` column with the predicted number of cases for that day, region and country. It is possible to leave `PredictedDailyNewCases` empty or NaN, or to remove the row, in case no predition is available.

In [10]:
EXAMPLE_OUTPUT_FILE = "../../../../2020-08-01_2020-08-04_predictions_example.csv"

In [11]:
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")

In [12]:
prediction_output_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases,IsSpecialty
0,Aruba,NaN,2020-08-01,0.820071,0
1,Aruba,NaN,2020-08-02,0.872854,0
2,Aruba,NaN,2020-08-03,0.000000,0
3,Aruba,NaN,2020-08-04,0.000000,0
4,Afghanistan,NaN,2020-08-01,80.590128,0


### Evaluation
Predictions will be evaluated on a period of 4 weeks **after** submision against the actual daily change in confirmed cases reported by the [Oxford COVID-19 Government Response Tracker (OxCGRT)](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker).

The latest data, including the latest confirmed cases ('ConfirmedCases') can be find here: https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv

In [13]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [14]:
df.sample(3)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
82584,United States,USA,Idaho,US_ID,STATE_TOTAL,2020-08-12,3.0,0.0,2.0,0.0,...,58.80,58.80,63.81,63.81,50.56,50.56,52.56,52.56,37.5,37.5
26394,Cameroon,CMR,NaN,NaN,NAT_TOTAL,2020-05-04,3.0,1.0,0.0,NaN,...,63.89,63.89,76.19,76.19,51.11,51.11,58.97,58.97,0.0,0.0
96328,Uzbekistan,UZB,NaN,NaN,NAT_TOTAL,2020-05-03,3.0,1.0,3.0,1.0,...,96.30,96.30,100.00,100.00,77.50,77.50,81.73,81.73,50.0,50.0


### Daily change in confirmed cases
The daily change in confirmed cases can be computed like this:

In [15]:
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)

For instance, for country **United States**, region **California**, the latest available changes in confirmed cases are:

In [16]:
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]

In [17]:
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
79515,United States,California,2020-12-16,1708559.0,63817.0
79516,United States,California,2020-12-17,1753974.0,45415.0
79517,United States,California,2020-12-18,1802195.0,48221.0
79518,United States,California,2020-12-19,1842557.0,40362.0
79519,United States,California,2020-12-20,NaN,0.0


# Training a model

## Copy the data locally

In [18]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])

RuntimeError: Physical devices cannot be modified after being initialized

In [1]:
# Reload the module to get the latest changes
# from xprize_predictor
# from importlib import reload
# reload(xprize_predictor)
from xprize_predictor import XPrizePredictor, EPOCHS

print(EPOCHS)

100


In [2]:
DATA_FILE = 'data/OxCGRT_latest.csv'
predictor = XPrizePredictor(None, DATA_FILE)

In [10]:
predictor_model = predictor.train()

Creating numpy arrays for Keras for each country...
Numpy arrays created
Trial 0
Epoch 1/100
167/167 [==============================] - 1s 8ms/step - loss: 0.8037 - val_loss: 0.0988
Epoch 2/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0800 - val_loss: 0.0718
Epoch 3/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0721 - val_loss: 0.0711
Epoch 4/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0702 - val_loss: 0.0770
Epoch 5/100
167/167 [==============================] - 1s 7ms/step - loss: 0.0694 - val_loss: 0.0686
Epoch 6/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0699 - val_loss: 0.0704
Epoch 7/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0677 - val_loss: 0.0686
Epoch 8/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0638 - val_loss: 0.0625
Epoch 9/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0613 - val_loss: 0.0636
Epoch 10/1

167/167 [==============================] - 1s 5ms/step - loss: 0.0522 - val_loss: 0.0565
Epoch 81/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0517 - val_loss: 0.0541
Epoch 82/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0523 - val_loss: 0.0557
Epoch 83/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0524 - val_loss: 0.0545
Epoch 84/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0518 - val_loss: 0.0543
Epoch 85/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0527 - val_loss: 0.0542
Epoch 86/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0522 - val_loss: 0.0544
Epoch 87/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0520 - val_loss: 0.0541
Epoch 88/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0519 - val_loss: 0.0541
Epoch 89/100
167/167 [==============================] - 1s 5ms/step - loss: 0.0520 - val_loss: 

In [9]:
from tensorflow.keras.utils import plot_model

plot_model(predictor.predictor)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [6]:
import os
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights("models/trained_model_weights.h5")

# Predicting using a trained model

## Load candidate model

In [ ]:
model_weights_file = "models/trained_model_weights.h5"

In [ ]:
predictor = XPrizePredictor(model_weights_file, DATA_FILE)

## Make prediction

In [ ]:
NPIS_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"

In [ ]:
%%time
preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

In [ ]:
preds_df.head()

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

In [ ]:
!head predictions/2020-08-01_2020-08-04.csv

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [ ]:
# Check the pediction file is valid
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")